## **Modelo CNN Simple - ResNet18 + Dropout + Soft Attention Espacial** 

In [1]:
import torch
import os
import pandas as pd
import seaborn as sns
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from datetime import datetime
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from PIL import Image

## Definición de transformación de los datos de entrada

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),  # Recorte aleatorio con zoom entre 80%-100%
    transforms.RandomHorizontalFlip(p=0.5),  # Voltea horizontalmente con 50% de probabilidad
    transforms.RandomRotation(15),  # Rotación aleatoria hasta 15 grados
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Ajuste aleatorio de color
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Pequeños desplazamientos aleatorios
    transforms.Resize((128, 128)),  # Redimensionar después de las transformaciones
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalización de ImageNet
])

val_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Solo redimensionar en validación (sin data augmentation)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


## Definición del dataset

In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        """
        Args:
            csv_file (str): Ruta al archivo CSV con las imágenes y sus etiquetas.
            img_dir (str): Ruta al directorio que contiene las imágenes.
            transform (callable, optional): Transformaciones que se aplican a las imágenes.
        """
        self.img_labels = pd.read_csv(csv_file)  # Leer el archivo CSV con las etiquetas
        self.img_dir = img_dir  # Ruta donde están las imágenes
        self.transform = transform  # Transformaciones a aplicar

        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.img_labels['diagnosis'].unique())}
    def __len__(self):
        """Retorna el número total de imágenes en el dataset"""
        return len(self.img_labels)

    def __getitem__(self, idx):
        """Obtiene una imagen y su etiqueta"""
        img_name = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])  # Nombre de la imagen
        image = Image.open(img_name)  # Abrir la imagen
        label = self.class_to_idx[self.img_labels.iloc[idx, 3]] # Etiqueta asociada

        if self.transform:
            image = self.transform(image)  # Aplicar transformaciones si es necesario

        return image, label

## Rutas de archivos

In [4]:
# Leer el CSV
csvRoute="./content/GPTeam-DeepLearning/Dataset/bcn_20k_train.csv"
imagesFolderRoute = "./content/GPTeam-DeepLearning/Dataset/bcn_20k_train/"
trainDataJsonRoute = "./content/GPTeam-DeepLearning/Dataset/data_train_resnet18_softAtt.json"
validationDataJsonRoute= "./content/GPTeam-DeepLearning/Dataset/data_val_resnet18_softAtt.json"
filteredCsvRoute = "./content/GPTeam-DeepLearning/Dataset/bcn_20k_train_filtrado.csv"
logsRoute = "./runs"
savedParametersRoute = "./saved_models_parameters/Parameters"
savedModelsRoute = "./saved_models_parameters/Models"
savedResultsRoute = "./results"

## Generación del dataset de entrenamiento y validación

***Carga del dataframe***

In [5]:
df = pd.read_csv(csvRoute)
df.head()

,bcn_filename,age_approx,anatom_site_general,diagnosis,lesion_id,capture_date,sex,split
0,BCN_0000000001.jpg,55.0,anterior torso,MEL,BCN_0003884,2012-05-16,male,train
1,BCN_0000000003.jpg,50.0,anterior torso,MEL,BCN_0000019,2015-07-09,female,train
2,BCN_0000000004.jpg,85.0,head/neck,SCC,BCN_0003499,2015-11-23,male,train
3,BCN_0000000006.jpg,60.0,anterior torso,NV,BCN_0003316,2015-06-16,male,train
4,BCN_0000000010.jpg,30.0,anterior torso,BCC,BCN_0004874,2014-02-18,female,train


***Exclusión de clases menos representativas***

In [6]:
#Definir las clases que deseas excluir por su nombre
clases_a_excluir = ['SCC', 'DF', 'VASC']  # Sustituye estos nombres por las clases que quieres excluir

# Filtrar el DataFrame para excluir las clases especificadas
df_filtrado = df[~df['diagnosis'].isin(clases_a_excluir)]

df_filtrado.to_csv(filteredCsvRoute, index=False)

df_filtrado['diagnosis'].unique()

df_original = df.copy()

df = df_filtrado.copy()

***Busqueda de spliteo adecuado (sin lesson_id iguales para dataset de entrenamiento y validación)***

In [7]:
# Obtener los valores únicos de lesion_id
unique_lesions = df['lesion_id'].unique()

# Búsqueda del mejor split
best_train, best_test = None, None
target_ratio = 0.8 * len(df)

for seed in range(1000):  # Exploramos diferentes random_state
    train_ids, test_ids = train_test_split(unique_lesions, test_size=0.2, random_state=seed)

    x_train = df[df['lesion_id'].isin(train_ids)]
    x_test = df[df['lesion_id'].isin(test_ids)]

    # Verificar que la cantidad de filas sea la correcta Y que los lesion_id no se repitan
    if abs(len(x_train) - target_ratio) < 1 and set(x_train['lesion_id']).isdisjoint(set(x_test['lesion_id'])):
        best_train, best_test = x_train, x_test
        print(f"Random state encontrado: {seed}")
        break

# Si no se encontró un split válido, lanzar error
if best_train is None or best_test is None:
    raise ValueError("No se encontró un split válido después de 1000 intentos")

# Asignar los mejores valores encontrados
train_df, val_df = best_train, best_test

assert set(train_df['lesion_id']).isdisjoint(set(val_df['lesion_id'])), "Error: Hay lesion_id repetidos entre train_df y val_df"

Random state encontrado: 25


In [8]:
# VERIFICACION ADICIONAL DE NO REPETICION DE LESSIONID EN AMBOS DATASETS
for idtrain in train_df['lesion_id'].unique():
    for idval in val_df['lesion_id'].unique():
        if idtrain.strip() == idval.strip():
            print("Coincidence found")

***Carga de las imágenes***

In [9]:
# Crear el dataset de entrenamiento y validación
train_dataset = CustomDataset(csv_file=filteredCsvRoute, img_dir=imagesFolderRoute, transform=train_transform)
val_dataset = CustomDataset(csv_file=filteredCsvRoute, img_dir=imagesFolderRoute, transform=val_transform)

# Actualizar los datasets con los subconjuntos correspondientes
train_dataset.img_labels = train_df
val_dataset.img_labels = val_df

In [10]:
# Verificación de la carga de todos los arhivos
archivos_en_directorio = []
# Recorrer todos los archivos en el directorio
for archivo in os.listdir(imagesFolderRoute):
    if os.path.isfile(os.path.join(imagesFolderRoute, archivo)):
        archivos_en_directorio.append(archivo)

# La columna del DataFrame con los nombres de los archivos
column_files = df['bcn_filename']
# Convertir las listas a conjuntos para realizar la diferencia
set_column_files = set(column_files)
set_archivos_en_drive = set(archivos_en_directorio)
valores_faltantes = set_column_files - set_archivos_en_drive

print(f"Cantidad de imagenes faltantes: {len(valores_faltantes)}")

Cantidad de imagenes faltantes: 0


## **Guardado de los datasets divididos inicialmente**

**Guardado de data de dataframes de entrenamiento y validación**

In [11]:
train_df.to_json(trainDataJsonRoute, orient='records', lines=True)
val_df.to_json(validationDataJsonRoute, orient='records', lines=True)

**Definición del Modelo**

In [12]:
# Mecanismo de Soft-Attention Espacial
class SpatialAttention(nn.Module):
    def __init__(self, in_channels):
        super(SpatialAttention, self).__init__()
        self.conv_attention = nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=1),  # Mapa de atención 1x1
            nn.Softmax(dim=2)                           # Normalización espacial
        )
        
    def forward(self, x):
        # x: (batch, 512, H, W) [Ej: (batch, 512, 7, 7)]
        # Generar mapa de atención (batch, 1, H, W)
        attn_weights = self.conv_attention(x)
        # Aplicar atención: características * pesos
        attended_features = x * attn_weights  # Broadcasting automático
        
        return attended_features


# Modelo ResNet18 con Dropout + Soft-Attention
class ResNet18WithAttention(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18WithAttention, self).__init__()
        # 1. Cargar ResNet18 preentrenado
        self.resnet18 = models.resnet18(pretrained=True)
        
        # 2. Congelar capas convolucionales
        for param in self.resnet18.parameters():
            param.requires_grad = False

        # 2.1 Descongelar la ultima capa
        for param in self.resnet18.layer4.parameters():
            param.requires_grad = True

        # 2.1 Descongelar la ultima capa
        for param in self.resnet18.layer3.parameters():
            param.requires_grad = True
        
        # 3. Añadir mecanismo de atención espacial
        self.attention = SpatialAttention(in_channels=512)  # ResNet18 tiene 512 canales al final
        
        # 4. Modificar capas FC con Dropout
        self.resnet18.fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # Extraer características hasta la última capa convolucional
        x = self.resnet18.conv1(x)
        x = self.resnet18.bn1(x)
        x = self.resnet18.relu(x)
        x = self.resnet18.maxpool(x)
        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)  # Salida: (batch, 512, 7, 7)
        
        # Aplicar atención espacial
        x = self.attention(x)  # (batch, 512, 7, 7) con pesos aprendidos
        
        # Global Average Pooling y clasificación
        x = self.resnet18.avgpool(x)  # (batch, 512, 1, 1)
        x = torch.flatten(x, 1)       # (batch, 512)
        x = self.resnet18.fc(x)       # (batch, num_classes)
        
        return x

# Viisualización de la ejecución del modelo

Ejecutar desde cmd lo siguiente para ver desde el navegador el menú de tensorboard: tensorboard --logdir=runs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

## **Entrenamiento y validación del modelo**

**Definición de gráficas adicionales a emplear**

In [13]:
# Función para registrar el heatmap (matriz de confusión)
def log_heatmap(writer, epoch, ground_truth, predictions, labels):
    # Calcula la matriz de confusión usando el orden de las etiquetas
    cm = confusion_matrix(ground_truth, predictions, labels=range(len(labels)))
    
    # Crear la figura
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_xlabel("Predicción")
    ax.set_ylabel("Groundtruth")
    ax.set_title("Matriz de Confusión")
    
    # Registrar la figura en TensorBoard
    writer.add_figure("Heatmap", fig, global_step=epoch)
    plt.close(fig)

# Función para registrar la grilla 2x2 de TP, FP, TN, FN (para clasificación binaria)
def log_values_predicted(writer, epoch, ground_truth, predictions):
    # Calcular la matriz de confusión
    cm = confusion_matrix(ground_truth, predictions, labels=[0, 1])
    
    # Crear anotaciones combinando etiqueta y valor
    annot = np.array([
        [f"TN\n{cm[0,0]}", f"FP\n{cm[0,1]}"],
        [f"FN\n{cm[1,0]}", f"TP\n{cm[1,1]}"]
    ])
    
    # Crear el heatmap
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=annot, fmt="", cmap="Blues",
                xticklabels=["False", "True"],
                yticklabels=["False", "True"],
                cbar=True, ax=ax)
    ax.set_xlabel("Eje Predicho (False/True)")
    ax.set_ylabel("Eje Verdadero (False/True)")
    ax.set_title("Predicción de valores")
    
    # Registrar la figura en TensorBoard
    writer.add_figure("Predicción de valores", fig, global_step=epoch)
    plt.close(fig)

**Seteo del dispositivo a usar**

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(F"Dispositivo utilizado: {device} ({torch.cuda.get_device_name()})")
else:
    print(F"Dispositivo utilizado: {device}")

Dispositivo utilizado: cuda (NVIDIA GeForce GTX 1650 SUPER)


**Especificación de parámetros**

In [15]:
learning_rate = 0.0001
num_classes = 5
num_epochs = 60
batch_size = 32

**Ejecución del modelo**

In [17]:
# Inicialización de tensorboard
executionModelDateTime = datetime.now().strftime("Ejecucion %d-%m-%Y %H-%M")
writer = SummaryWriter(log_dir=os.path.join(logsRoute,executionModelDateTime))

# Crear los DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Definición de la cantidad de clases, la función de perdida, el optimizador y el learning rate estático
modelAt = ResNet18WithAttention(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(modelAt.parameters(), lr=learning_rate, momentum=0.9,weight_decay=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
model = modelAt.to(device)
executionStatistics_list = []

for epoch in range(num_epochs):
    model.train()  # Modo entrenamiento
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    # Entrenamiento
    for inputs, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", leave=True):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Estadísticas de la pérdida
        running_loss += loss.item()
        # Precisión
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    #print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}, Accuracy: {100 * correct_preds / total_preds}%")
    train_loss = running_loss / len(train_dataloader)
    train_accuracy = 100 * correct_preds / total_preds

    # Validación
    model.eval()  # Modo evaluación
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    all_groundtruth = []
    all_predictions = []

    with torch.no_grad():  # No calcular gradientes durante la validación
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()  # Usamos correct_val en lugar de correct_preds
            total_val += labels.size(0)  # Usamos total_val en lugar de total_preds

            all_groundtruth.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    val_loss = running_val_loss/len(val_dataloader)
    val_accuracy = 100 * correct_preds / total_preds
    
    # Ajustar la tasa de aprendizaje si la pérdida de validación no mejora
    scheduler.step(val_loss)
    
    # Agregar valores a TensorBoard
    writer.add_scalar("Loss/Train", train_loss, epoch)
    writer.add_scalar("Loss/Validation", val_loss, epoch)
    writer.add_scalar("Accuracy/Train", train_accuracy, epoch)
    writer.add_scalar("Accuracy/Validation", val_accuracy, epoch)

    # Obtener los nombres de las clases (suponiendo que están en df_filtrados.columns)
    labels_names = list(df_filtrado['diagnosis'].unique())
    # Registrar el heatmap y la grilla de valores en TensorBoard
    log_heatmap(writer, epoch, all_groundtruth, all_predictions, labels_names)
    log_values_predicted(writer, epoch, all_groundtruth, all_predictions)
    
    # Carga de hiperparametros y metricas
    execution_statistics = {
        "epoch": epoch+1,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs,
        "Train Loss": train_loss,
        "Validation Loss": val_loss,
        "Train Accuracy": train_accuracy,
        "Validation Accuracy": val_accuracy
        
    }
    
    executionStatistics_list.append(execution_statistics)
    
    # Gráfica de loss y accuracy por cada epoca
    writer.add_scalars("Loss", {"Train": train_loss, "Validation": val_loss}, epoch)
    writer.add_scalars("Accuracy", {"Train": train_accuracy, "Validation": val_accuracy}, epoch)
    
    
    print(f"Epoch {epoch+1}/{num_epochs} FINISHED => "
        f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
        f"Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%")
    


writer.flush()
writer.close()

c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/60 - Training: 100%|██████████| 294/294 [04:07<00:00,  1.19it/s]


Epoch 1/60 FINISHED => Train Loss: 1.4022, Val Loss: 1.2332, Train Acc: 41.78%, Val Acc: 41.78%


Epoch 2/60 - Training: 100%|██████████| 294/294 [04:02<00:00,  1.21it/s]


Epoch 2/60 FINISHED => Train Loss: 1.2365, Val Loss: 1.1406, Train Acc: 52.98%, Val Acc: 52.98%


Epoch 3/60 - Training: 100%|██████████| 294/294 [04:04<00:00,  1.20it/s]


...

Epoch 58/60 FINISHED => Train Loss: 0.7957, Val Loss: 0.9297, Train Acc: 70.41%, Val Acc: 70.41%


Epoch 59/60 - Training: 100%|██████████| 294/294 [04:09<00:00,  1.18it/s]


Epoch 59/60 FINISHED => Train Loss: 0.7975, Val Loss: 0.9401, Train Acc: 69.76%, Val Acc: 69.76%


Epoch 60/60 - Training: 100%|██████████| 294/294 [04:09<00:00,  1.18it/s]


Epoch 60/60 FINISHED => Train Loss: 0.7983, Val Loss: 0.9386, Train Acc: 70.11%, Val Acc: 70.11%


# Guardado de los resultados del modelo

In [18]:
df_statistics = pd.DataFrame.from_dict(executionStatistics_list)
df_statistics.to_excel(os.path.join(savedResultsRoute, f"{executionModelDateTime}_results.xlsx"), index=False)

# Agregado de Hiperparámetros

# Guardado del modelo

**Guardado del modelo completo**

In [19]:
torch.save(model, os.path.join(savedModelsRoute, f"{executionModelDateTime}_modelo_entrenado_resnet18_softAtt_ka_completo.pth"))

**Guardado de los pesos del modelo**

In [20]:
torch.save(model.state_dict(), os.path.join(savedModelsRoute, f"{executionModelDateTime}_modelo_entrenado_resnet18_softAtt_ka_pesos.pth"))

In [21]:
# Mapeo de clases
print(train_dataset.class_to_idx)

{'MEL': 0, 'NV': 1, 'BCC': 2, 'BKL': 3, 'AK': 4}


# Prueba del modelo

In [23]:
model.eval()  # Ponemos el modelo en modo de evaluación

# Paso 3: Cargar la imagen y aplicar las transformaciones
image_path = 'ka.jpg'  # Pon aquí la ruta de tu imagen
image = Image.open(image_path)  # Abrir la imagen
image_tensor = val_transform(image)  # Aplicar las transformaciones

image_tensor = image_tensor.unsqueeze(0)  # Convertirlo a un batch de tamaño 1

# Paso 5: Mover la imagen al dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_tensor = image_tensor.to(device)
model = model.to(device)

# Paso 6: Realizar la predicción
with torch.no_grad():  # No necesitamos gradientes para la inferencia
    output = model(image_tensor)

# Paso 7: Convertir las predicciones en probabilidades con softmax
probabilities = torch.nn.functional.softmax(output, dim=1)  # Usamos dim=1 porque tenemos un batch

# Paso 8: Obtener la clase con la mayor probabilidad
_, predicted_class = torch.max(probabilities, dim=1)

# Paso 9: Interpretar la clase predicha
# Usamos el mapeo que ya tienes de clases (el 'class_to_idx' que ya definiste en tu dataset)
predicted_idx = predicted_class.item()  # Obtenemos el índice de la clase predicha
print(predicted_idx)
# Aquí usamos el mapeo de clases que creamos antes para convertir el índice a una clase legible
predicted_class_name = [key for key, value in train_dataset.class_to_idx.items() if value == predicted_idx][0]

# Mostrar la clase predicha
print(f"Predicción: {predicted_class_name}")

3
Predicción: BKL
